In [1]:
import pandas as pd
import os
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm

In [64]:
df=pd.read_feather("D://stata//BWdisclosure audits//2024 June cleaning analysis 7 countries//concat.feather")
CPCleanedUnique=list(df.CPCleaned.unique())

In [3]:
# df=pd.read_stata("D://stata//BWdisclosure audits//2024 June cleaning analysis 7 countries//June22cleaned.dta")
# df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())
# CPCleanedUnique=list(df.CPCleaned.unique())

C:\Users\Chunyun Li\AppData\Local\Temp\ipykernel_19740\2110774349.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())


In [58]:
def assign(q,orig:bool=False, cp="delete"):
    global df
    if orig==True:
        df.loc[df.Q==q,"CPCleaned"]=cp
    else:
        df.loc[df.QCleaned==q,"CPCleaned"]=cp
        
def getOriginalQ(cleanedQ):
    global df
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

def groupSent(sentences, threshold=0.75):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences,show_progress_bar=False)
    clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
    clustering.fit(embeddings)
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(sentences[sentence_id])
    return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

# with open("grpdQ.txt","r") as txt:
#     grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

def collapse(qcleaned, country):
    global df
    return df[(df.QCleaned==qcleaned) & (df.Country==country)].Q.value_counts()

def tab(cpnum=None,cp=None):
    assert cpnum!=None or cp!=None, "pass in at least 1 param."
    if cpnum!=None:
        cp=CPCleanedUnique[cpnum]
    print(cp)
    global df
    filtered_df=df[df['CPCleaned']==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def tabOcp(cporig=None):
    global df
    filtered_df=df[df['CP']==cporig]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()
#table for original CP

def tabOcpQ(cporig=None):
    global df
    filtered_df=df[df['CP']==cporig]
    return pd.crosstab(filtered_df['Country'], filtered_df['Q'],margins=True).transpose()


def discardYear(qcleaned, country, year:int=2021):
    global df
    df.loc[(df.QCleaned==qcleaned) & (df.Country==country) & (df.AssesmentDate.str[-4:]==str(year)),"CPCleaned"]="delete"

In [5]:
df.head(3)

,index,Country,FactoryAssessedID,AssesmentDate,QuestID,CP,Q,Finding,Tags,disclosed,CPCleaned,QCleaned,id
0,0,Vietnam,2457,27/05/2020,39390,other grounds,are hiv/aids tests required at hiring?,0,,0.0,other grounds,are hiv/aids tests required at hiring?,0
1,1,Vietnam,2556,02/01/2020,38002,other grounds,are hiv/aids tests required at hiring?,0,,0.0,other grounds,are hiv/aids tests required at hiring?,1
2,2,Vietnam,1419,20/09/2016,16656,other grounds,are hiv/aids tests required at hiring?,0,,0.0,other grounds,are hiv/aids tests required at hiring?,2


In [144]:
tab(cp='osh management systems')


osh management systems


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Assess OSH issues,584,2503,161,851,474,122,1802,6497
does the employer record work-related accidents?,584,2499,161,851,474,119,1802,6490
does the factory have a written osh policy?,584,2412,0,833,474,119,1802,6224
has legal construction permits/structural safety certificate,584,2354,161,851,474,119,1802,6345
mechanisms/committee for cooperation on osh,584,2503,161,851,474,122,1802,6497
All,2920,12271,644,4237,2370,601,9010,32053


In [67]:
replace(["has the employer done an initial industrial hygiene risk assessment, a risk map, and annual risk assessments?","has the employer performed an assessment of general occupational safety and health issues in the factory?"],"Assess OSH issues")
#combined Nicaragua item with others

In [68]:
replace(["has legal construction permits/structural safety certificate","does the employer have osh technical organizational regulations (tor) that are approved by the ministry of labor?"])
#combined one in Nicaragua with others

In [69]:
df.loc[(df.QCleaned=="does the factory have a valid business license?") & (df.Country=="Jordan"),"QCleaned"]="has legal construction permits/structural safety certificate"
##move one in Jordan to others

In [70]:
replace(["does the employer develop an osh plan annually?","does the employer have an osh license?"],"does the factory have a written osh policy?")
#move one item in Vietnam to others

In [71]:
replace(["does the factory employ the required osh specialists?","has the employer appointed an officer trained in firefighting, rescue and first aid?"])

In [72]:
discard("does the employer have an steam generator license?") #one unique in Nicaragua
discard("does the employer report both accidents and non-occurrences to the ministry of labor?") #one unique item in Nicaragua

In [73]:
replace(["has the employer set up a properly functioning osh collaborators network?"],"does the factory employ the required osh specialists?")
#move one in Vietnam to similar others on osh personnel

In [74]:
df.loc[(df.QCleaned=="does the factory have a valid business license?") & (df.Country=="Bangladesh"),"QCleaned"]="does the factory have a written osh policy?"
##move one in Bangladesh to others

In [75]:
for i in ["does the employer have the required certificates for the installation/ operation of electrical installations and other machines and equipment?",
       "does the employer inspect and measure the environmental conditions in the workplace on a annual basis?",
       "does the employer investigate work-related accidents and indicate the technical recommendations necessary to prevent them?",
       "does the employer regularly inspect and maintain machines, equipment, buildings and stores?",
       "has the employer designated a person exclusively devoted to osh promotion, prevention and protection?",
       "has the employer elaborated an osh training program?",
       "does the factory employ the required osh specialists?",
       "does the workplace have a functioning hiv/aids committee?"]:
    assign(i)


In [76]:
assign("does the employer comply with requirements on hiv/aids?")

In [77]:
getOriginalQ("mechanisms/committee for cooperation on osh")

Q
has the employer developed mechanisms to ensure cooperation between workers and management on osh matters?                                           2558
has the employer set up a properly functioning unit in charge of osh and/or osh council?                                                             1252
does the factory have a functioning safety committee?                                                                                                 584
has the employer set up a properly functioning unit in charge of osh and/or labour protection council?                                                550
does the factory have an osh committee ?                                                                                                              498
has the employer formed a joint worker/management osh committee?                                                                                      474
does the factory have an osh committee?                                   

In [78]:
collapse("mechanisms/committee for cooperation on osh","Cambodia")

Q
has the employer developed mechanisms to ensure cooperation between workers and management on osh matters?    2503
Name: count, dtype: int64

In [79]:
# df.loc[df.Q=="does the workplace have a functioning hiv/aids committee?", "QCleaned"]="does the workplace have a functioning hiv/aids committee?"

In [80]:
# export
#df.to_csv("D://audits621.csv", index=False)

In [145]:
tab(cp='wage information, use and deduction')

wage information, use and deduction


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
does the employer keep only one accurate payroll record?,584,2503,161,851,474,122,1802,6497
has the employer made any unauthorized deductions from wages?,584,2503,161,833,474,119,1802,6476
inform workers about wage pay and deduction,584,2503,161,0,474,122,1802,5646
All,1752,7509,483,1684,1422,363,5406,18619


In [82]:
replace(["does the employer keep only one accurate payroll record/ book?","does the employer keep only one accurate payroll record?"])

In [146]:
tab(cp="documentation and protection of young workers")

documentation and protection of young workers


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Keep a record of workers under age of 18,584,2237,161,833,474,119,1802,6210
Verify the age of workers before hiring,584,2503,161,851,0,122,1802,6023
All,1168,4740,322,1684,474,241,3604,12233


---

In [84]:
discard("does the employer remediate child labour as required under the cba?")
#99 deleted; one unique item asked in some years in Jordan

In [85]:
replace(["does the employer have a verification system in place to ensure they do not hire child labour in accordance with the national law?",
        "does the employer reliably verify the age of workers prior to hiring?",
         "does the employer cross-check worker's age and documents as stipulated by national law?"],"Verify the age of workers before hiring")

In [86]:
replace(["does the employer keep a record of workers under 18 years of age?",
         "does the employer comply with documentation requirements for workers under age of 18?",
         "does the employer have permission from the parents or legal guardians of workers between the ages of 14 and 16, and approval from the ministry of labor?"],
         "Keep a record of workers under age of 18")

In [87]:
discard("does the employer comply with requirements regarding the certificate of fitness for workers under age 18?")
#1 unique item in Bangladesh
discard("does the employer require workers under age 18 to provide a medical certificate, and an employment certificate or permit delivered by the director of labor?")
# 1 unique item in Haiti

In [147]:
tab(cp="overtime")

overtime


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Comply with legal limits on overtime hours,584,2503,161,851,0,122,1802,6023
Get permission/consent for overtime,584,2259,80,833,0,122,1802,5680
is overtime voluntary?,584,2503,161,833,474,119,1802,6476
All,1752,7265,402,2517,474,363,5406,18179


In [89]:
replace(["is overtime work limited to two hours per day?",
         "is overtime limited to three hours per day and nine hours per week?",
         "is overtime limited to 3 hours per day, 14 hours per week?",
         "is overtime in line with legal limits?",
         "does the employer comply with daily limits on overtime hours worked?	",
         "does the employer comply with limits on overtime hours worked?",
         "is overtime worked only for reasons allowed by law?"],"Comply with legal limits on overtime hours")

In [90]:
replace(["does the employer get permission from the labour inspector before workers work overtime?",
         "does the employer get written consent from women to work at night?","Get permission/consent for overtime"])

In [91]:
replace(["does the employer comply with yearly limits on overtime hours worked?","Get permission/consent for overtime"])
# combine one unique item in Vietnam with dissimilar others....

In [92]:
discard("does the employer provide compensatory days off when workers work on weekly rest days?") ##1 unique item in Bangladesh
discard("does the employer schedule overtime on public holidays only under conditions permitted by law?") # 2 strange entries in Nicaragua
discard("does the employer obtain authorization from the department of labour before working on sundays?") #only asked in 2015-2018 in Haiti
discard("does the employer get permission from the labor inspector before suspending weekly rest days?") # 1 item only asked in Camboia

In [93]:
df.loc[(df.QCleaned=="does the employer prepare written instructions on overtime?") & (df.Country=="Indonesia"),"QCleaned"]="Get permission/consent for overtime"
# move 1 unique item in Indonesia to others


In [94]:
for i in ["does the employer comply with monthly limits on overtime hours worked?",
           "does the employer comply with the monthly limits on overtime hours worked?"]:
    discard(i)
#one unique question in Vietnam

In [95]:
discard("does the employer prepare written instructions on overtime?") 

In [148]:
tab(cp="welfare facilities")

welfare facilities


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Adequate changing rooms/locker,584,2383,161,833,474,117,1802,6354
Adequate eating area,584,2368,161,833,474,122,1802,6344
adequate washing facilities,584,2503,161,851,474,122,1802,6497
does the workplace have adequate accessible toilets?,584,2279,161,833,474,122,1802,6255
providing drinking water,584,2503,161,851,474,122,1802,6497
All,2920,12036,805,4201,2370,605,9010,31947


In [97]:
replace(["does the workplace have adequate hand washing taps?","adequate washing facilities	"])

In [98]:
replace(["does the workplace provide all workers a place to store their clothes?",
         "does the workplace have adequate changing rooms/lockers to meet the needs of workers?",
         "does the workplace have adequate shelters or rest rooms?",
         "does the workplace have all required facilities?"],"Adequate changing rooms/locker")

In [99]:
replace(["does the workplace have an adequate lunch room, and/or canteen?",
         "does the workplace have an adequate eating area?",
         "does the employer comply with requirements regarding canteen?",
         "does employer provide cups or other sanitary means\nfor drinking water?"],
         "Adequate eating area")

In [100]:
replace(["does the workplace have a functioning and accessible nursing room?","Adequate changing rooms/locker"])
#combine 1 unique item in Cambodia with others

In [101]:
discard("does the employer keep food samples for 24 hours?")

In [102]:
df.loc[(df.QCleaned=="is the workplace clean and tidy?"),"CPCleaned"]="working environment"

In [103]:
discard("when provided as in-kind payment, does the employer give workers enough food of decent quality?") # 1 unique in Joradn
discard("does the workplace have adequate day care facilities?") #only asked in Bangladesh, Cambodia, and Jordan


In [154]:
tab(cp="health services and first aid")

health services and first aid


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Provide first aid training for workers,584,2286,161,0,474,119,1802,5426
Workers have medical checks as required by law,584,2386,161,834,474,122,1802,6363
does the employer address safety and health risks to pregnant or nursing workers?,584,2236,161,833,0,122,1802,5738
does the employer comply with requirements on hiv/aids?,584,2503,0,833,0,119,1802,5841
does the employer provide required medical services?,584,2503,161,833,474,0,1802,6357
has the employer ensured there are a sufficient number of readily accessible first aid boxes/supplies in the workplace?,584,2271,161,833,474,122,1802,6247
All,3504,14185,805,4166,1896,604,10812,35972


In [105]:
replace(["has the employer trained workers on first aid and formed a first-aid team?",
         "has the employer provided rescue and first-aid training for workers?",
         "has the employer provided first-aid training for workers?"],
         "Provide first aid training for workers")

In [106]:
replace(["do workers receive free periodic health checks as required by law?",
         "do workers have annual medical checks?",
         "do workers undergo a medical examination before being employed?"],
         "Workers have medical checks as required by law")

In [107]:
df.loc[df.Q.str.contains("annual medical checks"), "QCleaned"]="Workers have annual medical checks Haiti"

In [108]:
df.loc[df.Q=="does the workplace have a functioning hiv/aids committee?", "CPCleaned"]="health services and first aid"

In [109]:
df.loc[df.Q=="does the workplace have a functioning hiv/aids committee?", "QCleaned"]="does the employer comply with requirements on hiv/aids?"

In [110]:
getOriginalQ("does the employer comply with requirements on hiv/aids?")

Q
does the workplace have a functioning hiv/aids committee?                    2503
does the employer comply with the law on hiv/aids prevention and control?    1802
does the employer comply with requirements on hiv/aids?                       851
does the employer comply with national policy on hiv/aids?                    566
does the employer comply with requirements on hiv/aids ?                      119
Name: count, dtype: int64

In [152]:
df[df.Q=="does the employer comply with the law on hiv/aids prevention and control?"].CPCleaned.unique()

array(['delete'], dtype=object)

In [153]:
df.loc[df.QCleaned=="does the employer comply with requirements on hiv/aids?","CPCleaned"]="health services and first aid"

In [111]:
replace(["does the workplace have an adequately equipped and staffed infirmary?"],
        "does the employer provide required medical services?")
##move 1 in Cambodia to others

In [112]:
for i in ["does the workplace have adequate trained first aid officers?",
"does the employer provide adequate first-aid facilities?",
"do workers have a medical check within the first three months of hiring ?",
"Workers have annual medical checks Haiti"]:
    assign(i)

#unique items only in Haiti or Indonesia

In [155]:
tab(cp="union operations")

union operations


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
employer deducts union dues as requested,584,2240,161,833,474,119,1802,6213
employer provides facility for union,584,2503,0,833,474,0,1802,6196
union has access to workers,584,2503,161,851,474,122,1802,6497
All,1752,7246,322,2517,1422,241,5406,18906


In [156]:
tab(cp="regular hours")

regular hours


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Attendance records reflect hours actuallyed worked,584,2260,129,835,474,122,1802,6206
Comply with rest periods,584,2243,161,0,474,238,1802,5502
Regular daily work hours exceed legal limit,584,2503,161,851,474,120,1802,6495
All,1752,7006,451,1686,1422,480,5406,18203


In [115]:
replace(["do regular daily and weekly working hours exceed legal limits (40 hours per week; 7 hours a day, 6 days a week or 8 hours per day, 5 days a week)?",
         "do regular daily or weekly working hours exceed the legal limit (8 hours per day, 48 hours per week)?",
         "do regular daily or weekly working hours exceed the legal limit?",
         "do regular daily working hours exceed legal limits?",
         "do regular working hours exceed 10 hours per day, or 48 hours per week?",
         "do regular daily working hours exceed 10 hours?",
         "do regular working hours exceed 11 hours per day or 48 hours per week?"],
         "Regular daily work hours exceed legal limit")

In [116]:
replace(["do the attendance records reflect the hours actually worked?",
         "do the working time records reflect the hours actually worked?"],
         "Attendance records reflect hours actuallyed worked")

In [117]:
replace(["does the employer comply with daily break periods?",
         "does the employer provide required weekly rest periods?"],
         "Comply with rest periods")

In [441]:
for i in ["does the employer provide required breastfeeding breaks?",
          "does the employer post the factory's working hours (for all shifts, and including break times)?",
          "does the employer comply with shift work requirements?",
          "do workers have at least one day off per week?"]:
    assign(i)

In [440]:
df.loc[df.Q=="does the employer provide a weekly rest day after six consecutive days of work?",
        "QCleaned"]="do workers have at least one day off per week?"

In [119]:
discard("do regular work shifts exceed the daily legal limit?")

In [160]:
tab(cp="contracting procedures")

contracting procedures


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Comply with limits on probationary period,584,2241,161,833,474,0,1802,6095
Comply with requirements on subcontracted/outsourced workers,584,0,0,833,474,119,1802,3812
Limits on the use of fixed contracts or have contracts,584,2503,0,833,474,0,1802,6196
do workers pay any recruitment fees?,0,2236,0,833,474,0,0,3543
does the employer comply with requirements for temporary transfers of workers to new work?,0,0,0,0,0,0,1802,1802
"does the employer include the entire period of continuous employment when determining workers' entitlements to maternity leave, attendance bonus, seniority bonus, and/or annual leave?",0,2503,0,0,0,0,0,2503
All,1752,9483,161,3332,1896,119,7208,23951


In [121]:
replace(["do workers have to pay anyone to get a job?","do workers pay any recruitment fees?"])

In [122]:
replace(["does the employer comply with limits on the trial period for apprentices?",
         "is the probationary period applied only to permanent workers (pkwtt) and for no longer than 3 months?	",
         "is the probationary period limited to 3 months?",
         "Comply with limits on probationary period"])

In [123]:
replace(["does the employer comply with requirements concerning sub-contracted workers at the workplace?",
         "have you found non-compliance with legal requirements for compensation, contracts, osh and/or working time pertaining to non-production workers and/or sub-contracted workers?",
         "does the employer comply with legal requirements relating to outsourced workers?",
         "does the employer comply with national law concerning dispatched workers at the workplace?"],
         "Comply with requirements on subcontracted/outsourced workers")

In [124]:
replace(["limits on the use of fixed term contracts",
         "does the employer consider workers on limited duration contracts who work beyond their term as unlimited duration contract workers?",
         "do all workers (other than apprentices, badli or casual workers) have a service book?	"],
         "Limits on the use of fixed contracts or have contracts")

In [125]:
replace(["do the migrant workers have valid work permits and residence ids?"],
        "Comply with requirements on subcontracted/outsourced workers")
#move 1 in Jordan to others: on requirement for special type of workers

In [126]:
for i in ["has the employer taken sufficient steps to ensure that migrant workers do not pay any unauthorized fees?",
          "does the employer pay for the return trip of migrant workers who have been expelled from the country because of an invalid residence id?",
          "does the employer include the entire period of continuous employment when determining workers' entitlements to maternity leave, attendance bonus, seniority bonus, and/or annual leave?",
          "does the employer comply with the legal requirement relating to compensation for the non-permanent workers?",
          "does the employer comply with requirements for apprentices?",
          "does the employer comply with requirements concerning home workers?",
          "does the employer comply with legal requirements when subcontracting part of its work to another enterprise?"]:
    assign(i)

In [158]:
#df.to_feather("June22pm.feather")

In [159]:
df.drop(df[df.Q == "have you found non-compliance with legal requirements for compensation, contracts, osh and/or working time pertaining to non-production workers and/or sub-contracted workers?"].index, inplace=True)

In [127]:
tab(cp="bonded labour")

bonded labour


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
can workers who owe other types of debt to the employer freely leave their jobs?,584,2238,161,833,474,119,1802,6211
All,584,2238,161,833,474,119,1802,6211


In [265]:
tab(0)

other grounds


Country,Bangladesh,Cambodia,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,
Compliance on hiring disabled workers,0,2240,851,474,119,1802,5486
Disability a factor in conditions of work or termination,0,2236,833,0,119,1802,4990
has the employer made required accommodations for physically disabled workers?,0,2236,833,0,119,1802,4990
is disability a factor in decisions regarding pay?,584,2236,833,0,119,1802,5574
is there harassment of workers on the basis of disability?,0,2236,833,0,119,1802,4990
All,584,11184,4183,474,595,9010,26030


In [221]:
df.drop(df[df.Q == "does the employer allow workers with hiv/aids-related illnesses to work for as long as medically fit in available, appropriate work?"].index, inplace=True)
#2 entries in Nicaragua in 2015

In [243]:
replace(["are disabled persons who apply for work evaluated according to their ability to perform the job?",
        "compliance on hiring disabled workers"],
        "Compliance on hiring disabled workers")

In [241]:
df.loc[df.Q=="does the employer hire one disabled worker for every 100 workers?", "QCleaned"]="compliance on hiring disabled workers"

In [253]:
replace(["is disability a factor in decisions regarding conditions of work, access to training, promotion and pay?",
         "is disability a factor in decisions regarding pay?"])

In [260]:
replace(["is disability a factor in the employer's decisions regarding termination and retirement?",
         "is disability a factor in decisions regarding conditions of work?"],
         "Disability a factor in conditions of work or termination")

In [264]:
for i in ["is there harassment of workers on the basis of sexual orientation",
          "is there harassment of workers on the basis of age?",
          "is sexual orientation a factor in the employer's decisions regarding termination or retirement?",
          "is sexual orientation a factor in decisions regarding pay?",
          "is sexual orientation a factor in decisions regarding opportunities for promotion or access to training",
          "is sexual orientation a factor in decisions regarding conditions of work?",
          "is an applicant's sexual orientation a factor in hiring decisions?",
          "are a worker's family responsibilities a factor in the employer's decisions regarding termination or retirement of workers?",
          "are family responsibilities a factor in decisions regarding opportunities for promotion or access to training?",
          "are family responsibilities a factor in decisions regarding pay?",
          "are workers harassed on the basis of their family responsibilities?",
          "are workers harassed on the basis of their family responsibilities?",
          "do recruitment materials such as job announcements or job application forms refer to age in a manner that could discourage older workers from applying?",
          "is there harassment of workers on the basis of sexual orientation?",
          "is sexual orientation a factor in decisions regarding opportunities for promotion or access to training?",
          "is an applicant's family responsibilities a factor in hiring decisions?",
          "is age a factor in decisions regarding opportunities for promotion or access to training?",
          "are hiv/aids tests required at hiring?",
          "does the employer allow workers with aids-related illnesses to work for as long as medically fit in available, appropriate work?",
          "are hiv/aids tests required at hiring?",
          "does the employer allow workers with aids-related illnesses to work for as long as medically fit in available, appropriate work?",
          "has the employer taken legally required measures to help workers with hiv/ aids-related illnesses?",
          "is there harassment of workers on the basis of real or perceived hiv/aids status?",
          "is hiv/aids status a factor in decisions regarding pay?",
          "is hiv/aids status a factor in decisions regarding opportunities for promotion?",
          "is hiv/aids status a factor in decisions regarding conditions of work?",
          "is disability a factor that affects decisions about job promotion or access to training?",
          "s disability a factor in the employer's decisions regarding termination or retirement?",
          "is an applicant's real or perceived hiv/aids status a factor in hiring decisions?",
          "is an applicant's age a factor in hiring decisions?",
          "is age a factor in decisions regarding pay?",
          "is disability a factor in the employer's decisions regarding termination or retirement?",
          "is a worker's real or perceived hiv/aids status a factor in decisions regarding termination or retirement?"]:
    discard(i)

In [215]:
tab(12)

social security and other benefits


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Collect & forward workers' social security contribution,584,2503,161,851,474,122,1802,6497
Pay annual/seniority bonus,584,2503,161,851,474,122,1802,6497
"Pay other health,maternity, or group insurances",584,2503,161,851,474,122,1802,6497
Provide additional benefits,584,2250,161,834,0,122,0,3951
Provide other insurance or allowances,584,2503,161,833,0,0,0,4081
All,2920,12262,805,4220,1422,488,5406,27523


In [194]:
replace(["does the employer collect and forward workers' contributions to social insurance funds on time?",
         "does the employer correctly calculate, collect, and forward workers' contributions to the social security corporation?",
         "does the employer forward workers contributions to ofatma?",
         "collect and deposit workers' contributions into the fund",
         "does the employer pay the required employer contribution to the national social security fund?",
         "forward social insurance"],
         "Collect & forward workers' social security contribution")

In [208]:
replace(["Comply with requirements on healthcare fund",
         "does the employer provide compulsory group insurance for workers?",
         "pay for maternity insurance",
         "does the employer pay workers who are not covered by compulsory social insurance the employer's contribution for social, health and unemployment insurance?",
         "does the employer pay 2% of the gross payroll to inatec?",
         "does the employer pay all workers the correct mandatory wage supplements (including transportation and housing allowances)?",
         "does the employer correctly calculate and send the employer contribution to the social security corporation?"],
         "Pay other health,maternity, or group insurances")

In [210]:
replace(["does the employer provide round trip transport for female workers who work between 11 p.m. and 5 a.m?",
         "are the workers' participation and welfare funds used and distributed as required?",
         "does the employer pay 3% of workers' salary to ofatma for work-related accident insurance?",
         "does the employer pay the correct percentage of workers' total wages to the nicaraguan social security institute?"],
         "Provide additional benefits")

In [212]:
replace(["does the employer pay the required employer contribution to ona?",
         "does the employer provide nutritious food and drinks to female workers who work between 11 p.m. and 7 a.m?",
         "does the employer pay the required amount each year into the workers' participation fund?	",
         "does the employer pay all workers who work regularly the correct attendance bonus when workers take annual leave?"],
         "Provide other insurance or allowances")

In [163]:
discard("does the employer pay workers the correct compensation for work-related accidents and diseases?")
discard("does the employer pay the required amount into the sector-based central fund?")
# two items only asked since 2018 in Bangladesh

In [169]:
for i in ["do workers receive required distributions from the workers' participation fund?",
          "has the employer established a provident fund upon written demand by workers?",
          "if a provident fund has been established, does the employer match the contributions made by workers?"]:
    discard(i)
    ## three items in Bangaldesh only had 18 entries each in 2015

In [175]:
for i in ["does the employer pay all workers who work regularly the correct attendance bonus when workers take sick leave?",
          "does the employer pay all workers who work regularly the correct attendance bonus when workers take special leave?",
          "does the employer pay udc workers the correct seniority indemnity?"]:
    discard(i)
    #3 items inconsistent across years in Cambodia

In [165]:
replace(["does the employer comply with other wage payments?"],
        "does the employer pay workers two festival bonuses per year?")

In [177]:
df.loc[df.Q=="does the employer forward workers' contributions for social security to the nicaraguan social security institute?", 
       "QCleaned"]="does the employer collect and forward workers' contributions to social insurance funds on time?"
## recoded 2 entries in Nicaragua

In [179]:
df.loc[df.Q=="does the employer pay the correct percentage of workers' total wages to the nicaraguan social security institute?",
        "QCleaned"]="does the employer pay the correct percentage of workers' total wages to the nicaraguan social security institute?"
# recode one item in Nicaragua
df.loc[df.Q=="does the employer pay 16% of total wages to the nicaraguan social security institute?", 
       "QCleaned"]="does the employer pay the correct percentage of workers' total wages to the nicaraguan social security institute?"

In [181]:

df.loc[df.Q=="does the employer submit claims for sick leave and maternity leave to the social insurance agency within 10 days?",
        "QCleaned"]="does the employer submit claims for sick leave and maternity leave to the social insurance agency within 10 days?"

In [183]:
df.loc[df.Q=="does the employer submit claims for sick leave and maternity leave to the social insurance agency within 10 days?", "CPCleaned"]="paid leave"
#recode the CP of 27 wrongly assigned items in Vietnam

In [186]:
replace(["does the employer pay workers two festival bonuses per year?",
        "does the employer pay workers their annual salary supplement or bonus?",
        "does the employer pay workers the correct seniority bonus?",
        "does the employer pay workers a 13th month of wages?",],
        "Pay annual/seniority bonus")

In [206]:
replace(["does the employer comply with requirements when workers have occupational accidents and diseases?"],
        "Pay annual/seniority bonus")
##move 1 unique in Vietnam to others

In [196]:
df.loc[df.Q=="does the employer comply with all requirements related to national healthcare funds (bpjs kesehatan), including registering workers, paying employer contributions, and collecting and forwarding workers’ contribution?",
        "QCleaned"]="Comply with requirements on healthcare fund"
#recode one item in Indonesia

In [198]:
df.loc[df.Q=="does the employer pay the required contributions for social insurance (bpjs ketenagakerjaan) and health care funds (bpjs kesehatan) to bpjs?",
       "QCleaned"]="Comply with requirements on healthcare fund"
#recode one item in Indonesia

In [214]:
for i in ["does the employer forward workers contributions to ona?",
          "does the employer pay the correct attendance bonus to all new workers who start after the first of the month and to casual workers who work regularly?",
          "does the employer pay the correct compensation in the event of a worker's death?"]:
    discard(i)

In [266]:
tab(cp="chemicals and hazardous substances")

chemicals and hazardous substances


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are chemicals and hazardous substances properly labeled?,584,2503,161,0,0,122,1802,5172
does the employer keep an inventory of hazardous chemicals used in the workplace?,584,2293,161,833,474,119,1802,6266
does the employer keep chemical safety records for the hazardous chemicals used in the workplace?,584,2279,161,851,0,122,1802,5799
does the employer provide adequate washing facilities and cleaning materials in the event of exposure to hazardous chemicals?,584,2276,161,833,474,119,1802,6249
has the employer taken action to prevent and limit workers' exposure to chemicals and hazardous substances?,584,2290,161,834,474,122,1802,6267
has the employer trained all workers who work with chemicals and hazardous substances?,584,2285,161,833,0,122,1802,5787
storage of chemicals and hazardous substances,584,2503,161,851,474,122,1802,6497
All,4088,16429,1127,5035,1896,848,12614,42037


In [129]:
replace(["has the employer appointed a chemical officer?"],
        "has the employer taken action to prevent and limit workers' exposure to chemicals and hazardous substances?")
#move one unique item in Indonesia to others

In [130]:
df.loc[df.Q=="has the employer taken actions to assess, monitor, prevent and/or limit workers' exposure to hazardous chemicals?",
        "QCleaned"]="has the employer taken action to prevent and limit workers' exposure to chemicals and hazardous substances?"
#recode one item in Vietnam to the correct grouping

In [131]:
df.loc[df.Q=="has the employer taken actions to effectively assess, monitor, prevent and/or limit workers' exposure to hazardous chemicals?",
        "QCleaned"]="has the employer taken action to prevent and limit workers' exposure to chemicals and hazardous substances?"
#recode one item in Vietna to the correct grouping

In [132]:
replace(["are chemicals and hazardous substances properly stored?"],
        "storage of chemicals and hazardous substances")
#recode one item in Vietnam

In [133]:
discard("does the employer provide information on the chemicals and hazardous substances used in the workplace to the ministry of labour?")
#delete one unique item in Nicaragua

In [134]:
discard("does the employer have the legally required license for storage and use of acids in the factory?")
#1 unique in Bangladesh

In [135]:
discard("are chemical mixing rooms properly equipped and used?")
#1 unique in Cambodia

In [136]:
discard("has the employer prepared for response, control and cleaning of chemical spills?")
# 1 unique in Cambodia

In [137]:
tab(cp="child labourers")

child labourers


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
child labor,584,2503,161,851,474,122,1802,6497
All,584,2503,161,851,474,122,1802,6497


In [267]:
tab(cp="coercion")

coercion


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are workers free to terminate their employment with reasonable notice?,584,2238,161,833,474,119,1802,6211
does the employer delay or withhold wage payments in order to coerce workers to work?,584,2238,161,833,474,117,1802,6209
"does the employer deny workers access to their personal documents (such as birth certificates, passports, work permits, id card.) when they need them?",584,2236,161,833,474,119,1802,6209
does the employer force workers to work to discipline them or as punishment for participation in a strike?,584,2236,161,833,474,119,1802,6209
does the employer restrict workers' freedom to come and go from the dormitories and/or the industrial park or zone in which the factory is located?,584,2503,161,833,474,119,1802,6476
"does the employer use threats such as deportation, cancellation of visas or reporting to the authorities in order to force workers to stay at the job?",584,2236,161,833,474,119,1802,6209
does the employer use violence or the threat of violence to intimidate workers?,584,2236,161,833,474,119,1802,6209
free exit from workplace at all times,584,2503,126,851,474,122,1802,6462
All,4672,18426,1253,6682,3792,953,14416,50194


In [139]:
discard("does the employer require workers to work beyond the term of their contracts?")
#7 deleted; strange items in Haiti and Nicaragua

In [140]:
discard("does the employer use any other coercive tactics to overwhelm workers' ability to make decision in their own interest?")
# 1 item only asked in 3 countries

In [141]:
tab(cp="collective bargaining")

collective bargaining


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
cba at least as favorable as law,584,2503,161,851,0,122,1802,6023
employer consult with workers/union,584,2503,0,851,0,122,1802,5862
employer inform workers cba,584,2236,161,833,474,119,1802,6209
employer refused to bargain in good faith,584,2503,161,851,474,122,1802,6497
implement cba,584,2503,161,851,474,122,1802,6497
All,2920,12248,644,4237,1422,607,9010,31088


In [268]:
tab(cp="contracting procedures")

contracting procedures


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Comply with limits on probationary period,584,2241,161,833,474,0,1802,6095
Comply with requirements on subcontracted/outsourced workers,584,0,0,833,474,119,1802,3812
Limits on the use of fixed contracts or have contracts,584,2503,0,833,474,0,1802,6196
do workers pay any recruitment fees?,0,2236,0,833,474,0,0,3543
does the employer comply with requirements for temporary transfers of workers to new work?,0,0,0,0,0,0,1802,1802
"does the employer include the entire period of continuous employment when determining workers' entitlements to maternity leave, attendance bonus, seniority bonus, and/or annual leave?",0,2503,0,0,0,0,0,2503
All,1752,9483,161,3332,1896,119,7208,23951


In [143]:
df.loc[df.Q=="does the employer include the entire period of continuous employment when determining workers' entitlements to maternity leave, attendance bonus, seniority bonus, and/or annual leave?", "CPCleaned"]="contracting procedures"

In [283]:
tab(19)

termination


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Comply with legal requirements before suspending workers,584,2503,156,0,474,122,1802,5641
Pay correct severance pay,584,2503,161,851,474,122,1802,6497
Terminate workers for just cause,584,2503,161,851,474,122,1802,6497
does the employer compensate workers for unused paid annual leave when they resign or are terminated?,584,2503,161,851,474,0,1802,6375
does the employer provide workers proper notice of termination?,584,2503,161,0,474,0,1802,5524
All,2920,12515,800,2553,2370,366,9010,30534


In [270]:
replace(["does the employer only terminate workers for valid reasons?",
         "does the employer get ministry of labour authorization prior to terminating workers for just cause?"],
         "Terminate workers for just cause")

In [272]:
replace(["does the employer pay workers the correct severance pay?",
         "does the employer pay workers their annual salary supplement or bonus upon termination?"],
         "Pay correct severance pay")

In [282]:
for i in ["does the employer terminate workers for reasons that are prohibited by law?",
          "does the employer unilaterally terminate workers for reasons that are not allowed by law?",
          "do resigned or terminated workers receive all other termination benefits required by law?",
          "has the employer complied with any orders to reinstate or compensate workers who were found to be unjustly terminated?",
          "does the employer provide terminated workers their provident funds (if applicable)?",
          "does the employer pay workers the correct indemnity for dismissal?",
          "does the employer pay terminated workers the correct damages when required?",
          "are workers given the opportunities required under law to defend themselves before they are terminated?",
          "does the employer pay all outstanding wages and indemnities within 48 hours of termination?"]:
    discard(i)

In [276]:
replace(["does the employer comply with legal requirements before suspending workers due to changes in operations?",
         "does the employer notify the chief inspector and the collective bargaining agent before retrenching workers due to redundancy?"],
         "Comply with legal requirements before suspending workers")
#move one similar item in Bangladesh to others

In [321]:
tab(24)

employment contracts


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Company bylaws comply with law,584,2265,161,834,474,119,1802,6239
Employment contracts comply with law,584,2286,161,833,474,119,1802,6259
Employment contracts specify terms of employment,584,2340,161,833,474,121,1802,6315
Provide workers a employment contract,584,2503,161,833,474,0,1802,6357
do workers understand the terms and conditions of employment?,584,2330,161,0,474,119,1802,5470
All,2920,11724,805,3333,2370,478,9010,30640


In [ ]:
replace(["do the employment contracts comply with cambodian labour law and with the factory's internal regulations?",
         "do the employment contracts comply with jordanian legal requirements?",
         ""])

In [287]:
getOriginalQ("do the employment contracts comply with other legal requirements?")

Q
do the employment contracts specify the terms and conditions of employment?            4621
do the work agreements specify the terms and conditions of employment?                  833
do the written employment contracts specify the terms and conditions of employment?     156
do the employment contracts comply with other legal requirements?                       117
does the employer comply with requirements for migrant workers' contracts?                2
Name: count, dtype: int64

In [308]:
getOriginalQ("Provide workers a copy of employment contract")

Q
do all persons who perform work for the factory, both on the premises and offsite, have a contract?    4940
does the employer give a copy of the work agreement in bahasa to workers?                               833
has the employer provided all workers with a letter of appointment?                                     584
does the employer provide workers with a copy of their contract?                                        474
Name: count, dtype: int64

In [293]:
df.loc[df.Q=="do the employment contracts specify the terms and conditions of employment?",
        "QCleaned"]="Employment contracts specify terms of employment"

df.loc[df.Q=="do the work agreements specify the terms and conditions of employment?",
        "QCleaned"]="Employment contracts specify terms of employment"

df.loc[df.Q=="do the written employment contracts specify the terms and conditions of employment?",
        "QCleaned"]="Employment contracts specify terms of employment"

df.loc[df.Q=="do the letters of appointment specify the terms and conditions of employment?",
        "QCleaned"]="Employment contracts specify terms of employment"

df.loc[df.Q=="do the employment contracts specify the terms and conditions of employment relation? ",
        "QCleaned"]="Employment contracts specify terms of employment"

In [299]:
replace(["do the employment contracts comply with cambodian labour law and with the factory's internal regulations?",
         "do the employment contracts comply with jordanian legal requirements?",
         "do the letters of appointment comply with the labour law, collective agreement and service rules?",
         "do the employment contracts comply with other legal requirements?",
         "do the contracts comply with the labour law, collective agreement and work rules?"],
         "Employment contracts comply with law")

In [297]:
df.loc[df.Q=="do the work agreements comply with company regulations, the collective labour agreement, and prevailing laws and regulations?", 
       "QCleaned"]="Employment contracts comply with law"
#one similar item in Indonesia

In [301]:
replace(["has the employer provided all workers with a letter of appointment?",
         "does the employer provide workers with a copy of their contract?",
         "does the employer give a copy of the work agreement in bahasa to workers?"],
         "Provide workers a copy of employment contract")

In [303]:
replace(["do the factory bylaws comply with jordanian legal requirements and were they communicated to workers?",
         "do the internal work rules comply with national law?",
         "do the service rules comply with legal requirements?"],
         "Company bylaws comply with law")

In [304]:
df.loc[df.Q=="do the company regulations comply with legal requirements as stipulated in the labour law and regulations?",
        "QCleaned"]="Company bylaws comply with law"
#one similar item in Indonesia

In [309]:
df.loc[df.Q=="do all persons who perform work for the factory, both on the premises and offsite, have a contract?",
        "QCleaned"]="All workers have employment contracts"

In [314]:
df.loc[(df.Q=="do all persons who perform work for the factory, both on the premises and offsite, have a contract?")
        & (df.Country=="Jordan"),"QCleaned"]="Workers on premise and offsite have a contract"

In [316]:
replace(["All workers have employment contracts",
         "Provide workers a copy of employment contract"],
         "Provide workers a employment contract")

In [320]:
discard("Workers on premise and offsite have a contract")
discard("does the employer maintain a personnel file for each worker?")

In [335]:
tab(cp="dialogue, discipline and disputes")

dialogue, discipline and disputes


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
bullying harassment,584,2503,161,851,474,122,1802,6497
did the employer resolve grievances and disputes in compliance with legal requirements?,584,2503,161,833,474,0,1802,6357
do disciplinary measures comply with legal requirements?,584,2236,0,833,474,119,1802,6048
have participation committee or shop steward,584,2411,0,851,474,0,1802,6122
All,2336,9653,322,3368,1896,241,7208,25024


In [328]:
discard("if the ministry of labor has delivered a resolution about the suspension or temporary closure, were all the provisions of the resolution met?")
discard("did the factory consult the measures (suspension or temporary closed) to be taken with the workers' representatives? \ndid they reach an agreement?\ndid you present the agreement to the ministry of labor?")
## two items only asked 3 times in Nicaragua 

In [330]:
getOriginalQ("have participation committee or shop steward")

Q
does the employer comply with requirements regarding shop stewards?                                   2411
does the factory have a functioning bipartite cooperation institution?                                 851
does the factory have a functioning participation committee?                                           584
does the factory have a functioning joint union-management committee?                                  375
if union labour committee is formed, does employer meet worker representatives on a regular basis?      99
Name: count, dtype: int64

In [331]:
getOriginalQ("does dialogue at the workplace take place in compliance with legal requirement?")

Q
does dialogue at the workplace take place in compliance with legal requirement?    1802
Name: count, dtype: int64

In [332]:
replace(["have participation committee or shop steward",
         "does dialogue at the workplace take place in compliance with legal requirement?"])

In [334]:
for i in ["has the employer complied with final conciliation agreements?",
          "has the employer complied with court orders arising from individual or collective disputes?",
          "has the employer complied with arbitration awards?",
          "does the factory have a qualified welfare officer?"]:
    discard(i)

In [336]:
tab(cp="documentation and protection of young workers")

documentation and protection of young workers


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Keep a record of workers under age of 18,584,2237,161,833,474,119,1802,6210
Verify the age of workers before hiring,584,2503,161,851,0,122,1802,6023
All,1168,4740,322,1684,474,241,3604,12233


In [339]:
tab(cp="emergency preparedness")

emergency preparedness


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
accessable unlocked emergency exits,584,2503,161,833,474,119,1802,6476
are emergency exits and escape routes clearly marked?,584,2296,161,833,474,119,1802,6269
are flammable materials safely stored?,584,2254,161,833,474,117,1802,6225
are possible sources of ignition appropriately safeguarded?,584,2241,161,833,474,117,1802,6212
at least 2 possible emergency exits,584,2503,161,833,474,119,1802,6476
does the workplace have adequate fire-fighting equipment?,584,2503,161,833,474,119,1802,6476
fire detection and alarm system,584,2503,161,833,474,119,1802,6476
has the employer appointed and trained a fire management team?,584,2503,155,833,474,119,1802,6470
periodic emergency drills,584,2503,161,833,474,119,1802,6476


In [338]:
for i in ["has the employer elaborated and implemented an emergency plan?",
          "does the workplace have at least one adequate window exit per floor?",
          "does the workplace have at least one adequate window exit per floor?",
          "does the workplace have adequate stairways that can be used in case of emergency?",
          "does the employer comply with size requirements for escape routes?",
          "does the employer comply with emergency requirements?",
          "are the emergency exits locked during working hours, including overtime?",
          "are floors and corridors in line with legal requirements?",
          "are escape routes free of obstruction?"]:
    assign(i)

In [340]:
tab(cp="forced labour and overtime")

forced labour and overtime


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
forced overtime under threat of penalty,584,2503,161,851,474,122,1802,6497
All,584,2503,161,851,474,122,1802,6497


In [345]:
tab(cp="freedom to associate")

freedom to associate


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
employer requires workers to join union,584,2503,161,851,474,122,1802,6497
All,584,2503,161,851,474,122,1802,6497


In [344]:
discard("freedom to form/join union")

In [362]:
tab(cp="gender")

gender


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
"does the employer change the employment status, position, wages, benefits or seniority of workers during maternity leave?",584,2236,161,833,0,122,1802,5738
gender a factor in promotion,584,2503,161,833,0,119,1802,6002
gender a factor in termination,584,2503,161,851,0,122,1081,5302
gender a factor regarding pay,584,2503,161,833,474,119,1802,6476
gender discrimination on conditions of work,584,2503,161,851,474,122,1802,6497
is an applicant's gender a factor in hiring decisions?,584,2503,161,851,0,122,1802,6023
job announcements refer to gender,584,2503,161,0,474,119,1802,5643
maternity leave included in continous service,584,2236,161,833,474,119,1802,6209
requires pregnancy test/contraceptive,584,2236,161,851,474,119,1802,6227


In [354]:
df.drop(df[(df.QCleaned=="gender a factor in promotion") & (df.Country=="Haiti")].index,inplace=True)

In [349]:
getOriginalQ("maternity a factor in termination")

Q
does the employer terminate workers who are on maternity leave or force them to resign?                             2503
is gender or marital status a factor in the employer's decisions regarding termination or retirement of workers?     161
Name: count, dtype: int64

In [358]:
df.loc[(df.QCleaned=="maternity a factor in termination") & (df.Country=="Haiti"),"QCleaned"]="gender a factor in promotion"
#move one item in Haiti to others to keep it 

In [361]:
discard("maternity a factor in termination")

In [367]:
tab(cp="hazardous work and other worst forms")

hazardous work and other worst forms


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Workers under 18 subject to worst forms of child labour,584,2236,161,833,474,119,1802,6209
are any workers who are under age 18 working at night?,584,2236,161,833,474,119,1802,6209
do workers under the age of 18 perform work that is hazardous by nature?,584,2236,161,833,0,119,1802,5735
All,1752,6708,483,2499,948,357,5406,18153


In [366]:
replace(["are any workers who are under age 18 working overtime?",
         "are any workers under age 18 subjected to the unconditional worst forms of child labour?"],
         "Workers under 18 subject to worst forms of child labour")

In [368]:
tab(cp="health services and first aid")

health services and first aid


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Provide first aid training for workers,584,2286,161,0,474,119,1802,5426
Workers have medical checks as required by law,584,2386,161,834,474,122,1802,6363
does the employer address safety and health risks to pregnant or nursing workers?,584,2236,161,833,0,122,1802,5738
does the employer comply with requirements on hiv/aids?,584,2503,0,833,0,119,1802,5841
does the employer provide required medical services?,584,2503,161,833,474,0,1802,6357
has the employer ensured there are a sufficient number of readily accessible first aid boxes/supplies in the workplace?,584,2271,161,833,474,122,1802,6247
All,3504,14185,805,4166,1896,604,10812,35972


In [369]:
tab(cp="interference and discrimination")

interference and discrimination


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
employer disuades workers from unions,584,2503,161,833,474,119,0,4674
employer fires union official,584,2503,0,833,474,122,1802,6318
employer fires workers for union activity,584,2503,156,851,474,121,1802,6491
employer punishes workers for union activity,584,2503,161,839,474,122,1802,6485
employer threatens union members,584,2503,161,833,474,122,1802,6479
interfere union,566,2503,161,851,474,122,1802,6479
union membership/activity influences hiring,584,2503,161,833,474,121,1802,6478
workers are free to meet,584,2237,161,833,474,119,1802,6210
All,4654,19758,1122,6706,3792,968,12614,49614


In [437]:
tab(cp="leave")

leave


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Give workers special leave,584,2503,161,833,0,122,1802,6005
does the employer provide required annual leave?,584,2503,161,851,474,122,1802,6497
does the employer provide required maternity leave?,584,2503,161,851,474,122,1802,6497
does the employer provide required sick leave?,584,2503,161,851,474,122,1802,6497
All,2336,10012,644,3386,1422,488,7208,25496


In [371]:
replace(["does the employer provide required time off for paternity leave?",
         "does the employer provide other types of required leave?"],
         "Provide paternity leave")
#integrate 1 item in Nicaragua

In [373]:
replace(["does the employer give workers up to 7 days of time off for special leave per year?",
         "does the employer provide 10 days of casual leave per year?",
         "does the employer provide required personal leave?",
         "does the employer provide at least 15 days of annual leave to workers with 1 year of service?"],
         "Give workers special leave")

In [378]:
df.loc[(df.QCleaned=="period break") & (df.Country=="Indonesia"),"QCleaned"]="Give workers special leave"

In [382]:
for i in ["are pregnant workers provided time off for prenatal visits?",
          "does the employer provide 1 hour off to elderly workers in their last year of work before retirement?",
          "does the employer provide workers 11 festival holidays per year (or compensatory and substitute holidays when workers work on festival holidays)?",
          "if workers are paid for annual leave instead of receiving time off, is there a valid agreement between workers and management that provides for a portion of the leave to be paid?",
          "period break",
          "Provide paternity leave"]:
    discard(i)

In [436]:
discard("does the employer provide required breastfeeding breaks?")

In [390]:
tab(cp="method of payment")

method of payment


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are wages paid on time?,584,2294,161,851,474,122,1802,6288
All,584,2294,161,851,474,122,1802,6288


In [385]:
replace(["do in-kind wage payments comply with national law?",
        "are wages paid directly to workers?"])
#move one item in Jordan to the latter to keep Jordan item

In [389]:
discard("are wages paid directly to workers?")
discard("are workers' cash wages paid in legal currency?")

In [393]:
tab(cp="minimum wages/piece rate wages")

minimum wages/piece rate wages


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are piece-rate workers paid correctly for ordinary hours of work?,584,2503,126,833,474,119,1802,6441
minimum pay to temporary workers,584,2503,161,833,474,121,1802,6478
minimum wages for regular workers,584,2503,161,851,474,122,1802,6497
All,1752,7509,448,2517,1422,362,5406,19416


In [392]:
for i in ["minimum pay to apprentices",
          "is the piece rate set at a level that permits workers of average ability working normal hours to earn minimum wage?",
          "does the employer pay at least 7% higher than the applicable legal minimum wage for ordinary hours of work to workers who have received vocational training?",
          "does the employer pay apprentices the wage agreed to by the parties?",
          "does the employer establish a wage structure and wage scale, announce it to all workers, and submit it to the local manpower office?"]:
    discard(i)

In [394]:
tab(cp="osh management systems")

osh management systems


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Assess OSH issues,584,2503,161,851,474,122,1802,6497
does the employer record work-related accidents?,584,2499,161,851,474,119,1802,6490
does the factory have a written osh policy?,584,2412,0,833,474,119,1802,6224
has legal construction permits/structural safety certificate,584,2354,161,851,474,119,1802,6345
mechanisms/committee for cooperation on osh,584,2503,161,851,474,122,1802,6497
All,2920,12271,644,4237,2370,601,9010,32053


In [397]:
tab(cp="overtime")

overtime


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Comply with legal limits on overtime hours,584,2503,161,851,0,122,1802,6023
Get permission/consent for overtime,584,2259,0,833,0,122,1802,5600
is overtime voluntary?,584,2503,161,833,474,119,1802,6476
All,1752,7265,322,2517,474,363,5406,18099


In [396]:
df.drop(df[(df.QCleaned=="Get permission/consent for overtime") & (df.Country=="Haiti")].index,inplace=True)

In [405]:
tab(cp="overtime wages")

overtime wages


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
correct pay for overtime at weekly rest days,584,2503,161,834,0,119,1802,6003
correct pay for work on public holidays etc,584,2503,161,0,474,119,1802,5643
payment of overtime,584,2503,161,851,474,122,1802,6497
All,1752,7509,483,1685,948,360,5406,18143


In [425]:
tab(cp="paid leave")

paid leave


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Paid for special leave,0,2503,161,839,0,122,1802,5427
does the employer pay workers correctly for paid public holidays?,584,2243,0,851,474,119,1802,6073
legal pay for annual leave,584,2503,161,851,474,122,1802,6497
paid leave for breastfeed,584,2259,161,833,474,119,1802,6232
paid sick leave,584,2503,0,851,474,122,1802,6336
pay for work stoppage,584,2503,0,851,474,122,1802,6336
payment for maternity leave,584,2503,161,851,474,120,1802,6495
All,3504,17017,644,5927,2844,846,12614,43396


In [407]:
replace(["does the employer pay workers correctly during casual leave?"],
        "paid leave for breastfeed")
#move former 1 in Bangladesh to others to keep the Bangladesh item

In [415]:
df.drop(df[(df.Q =="does the employer settle claims for sick leave and maternity leave within 3 working days?")].index, inplace=True)
#247 entries, only asked in some years in Vietnam

In [431]:
getOriginalQ("paid leave for breastfeed")

Q
does the employer pay workers for one hour breastfeeding break per day?                                                                                     1802
does the employer provide workers one hour of paid time off per day for breast-feeding?                                                                     1153
does the employer pay workers correctly for the one hour of paid time off per day for breastfeeding?                                                        1106
does the employer pay workers correctly during breastfeeding breaks, as required under the work agreements, company regulations or collective agreement?     833
does the employer pay workers correctly during breastfeeding breaks?                                                                                         640
does the employer pay workers correctly during casual leave?                                                                                                 584
does the employer pay workers co

In [417]:
replace(["do workers receive correct payment during sick leave and other types of leave that are covered by social insurance?",
         "paid sick leave"])

In [419]:
replace(["does the employer pay workers correctly for weekly rest days?",
         "does the employer pay any workers incorrectly for any types of paid time off (breaks and leave)?",
         "does the employer pay workers correctly during special leave?",
         "does the employer pay workers correctly for personal leave?",
         "does the employer pay workers correctly when they take time off for personal/ family matters, as required by national law?"],
         "Paid for special leave")

In [421]:
for i in ["do pregnant workers receive correct payment when they take time off for prenatal care?",
          "does the employer pay for paternity leave when required?",
          "does the employer pay women workers for 30 minutes rest per day during their periods?",
          "does the employer pay workers correctly during daily breaks?",
          "does the employer pay workers correctly for non-working weekly rest days?",
          "does the employer pay workers correctly when they feel pain during the first two days of menstruation and they notify the employer?",
          "does the employer pay workers who are subpoenaed to testify as witness, or as plaintiff or respondents in court cases or administrative hearings?",
          "does the employer provide 1 hour of paid time off to elderly workers in their last year of work before retirement?",
          "does the employer submit claims for sick leave and maternity leave to the social insurance agency within 10 days?"]:
    discard(i)

In [423]:
df.drop(df[(df.QCleaned=="does the employer pay workers correctly for paid public holidays?") & (df.Country=="Haiti")].index,inplace=True)
#133 deleted; not asked since 2020 in Haiti

In [424]:
df.drop(df[(df.QCleaned=="paid sick leave") & (df.Country=="Haiti")].index,inplace=True)
#142 deleted; not asked in 2016 in Haiti

In [427]:
tab(cp="prison labour")

prison labour


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
prison labor consent and same treatment,584,2238,161,833,474,119,1802,6211
All,584,2238,161,833,474,119,1802,6211


In [428]:
tab(cp="race and origin")

race and origin


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
"are workers harassed on the basis of their race, colour or origin?",584,2503,161,833,474,119,1802,6476
"do recruitment materials such as job announcements or job application forms refer to the applicant's race, colour or origin?",584,2503,161,0,474,119,1802,5643
"is an applicant's race, colour or origin a factor in hiring decisions?",584,2503,161,833,0,119,1802,6002
"is race, colour or origin a factor in decisions regarding conditions of work?",584,2503,161,0,0,119,1802,5169
"is race, colour or origin a factor in decisions regarding opportunities for promotion or access to training?",584,2503,161,833,474,119,1802,6476
"is race, colour or origin a factor in decisions regarding pay?",584,2503,161,0,474,119,1802,5643
"is race, colour or origin a factor in the employer's decisions regarding termination or retirement of workers?",584,2503,161,833,0,119,1802,6002
All,4088,17521,1127,3332,1896,833,12614,41411


In [442]:
tab(cp="regular hours")

regular hours


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Attendance records reflect hours actuallyed worked,584,2260,0,835,474,122,1802,6077
Comply with rest periods,584,2243,161,0,474,119,1802,5383
Regular daily work hours exceed legal limit,584,2503,161,851,474,120,1802,6495
All,1752,7006,322,1686,1422,361,5406,17955


In [432]:
df.loc[df.Q=="does the employer provide required breastfeeding breaks?", "CPCleaned"]="leave"

In [438]:
df.drop(df[(df.QCleaned=="Attendance records reflect hours actuallyed worked") & (df.Country=="Haiti")].index,inplace=True)
#129 deleted, not asked in 2019 in Haiti

In [443]:
tab(cp="religion and political opinion")

religion and political opinion


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are workers harassed on the basis of their religion or political opinion?,584,2503,161,833,474,119,1802,6476
do recruitment materials such as job announcements or job application forms refer to the applicant's religion or political opinion?,584,2503,161,0,0,119,1802,5169
is an applicant's religion or political opinion a factor in hiring decisions?,584,2503,161,833,474,119,1802,6476
is religion or political opinion a factor in decisions regarding conditions of work?,584,2503,161,0,0,119,1802,5169
is religion or political opinion a factor in decisions regarding pay?,584,2503,161,0,474,119,1802,5643
is religion or political opinion a factor in the employer's decisions regarding termination or retirement of workers?,584,2503,161,833,0,119,1802,6002
religion a factor of training or promotion,584,2503,161,833,474,119,1802,6476
All,4088,17521,1127,3332,1896,833,12614,41411


In [444]:
tab(cp="strikes")

strikes


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
employer called police to break up strike,584,2236,161,833,474,119,1802,6209
employer prevented workers from striking,584,2236,161,833,474,122,1802,6212
employer punished workers for striking,584,2503,161,851,474,122,1802,6497
employer replaced striking workers,584,2236,161,833,474,122,1802,6212
All,2336,9211,644,3350,1896,485,7208,25130


In [446]:
tab(cp="union operations")

union operations


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
employer deducts union dues as requested,584,2240,161,833,474,119,1802,6213
employer provides facility for union,584,2503,0,833,474,0,1802,6196
union has access to workers,584,2503,161,851,474,122,1802,6497
All,1752,7246,322,2517,1422,241,5406,18906


In [447]:
tab(cp="wage information, use and deduction")

wage information, use and deduction


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
does the employer keep only one accurate payroll record?,584,2503,161,851,474,122,1802,6497
has the employer made any unauthorized deductions from wages?,584,2503,161,833,474,119,1802,6476
inform workers about wage pay and deduction,584,2503,161,0,474,122,1802,5646
All,1752,7509,483,1684,1422,363,5406,18619


In [448]:
tab(cp="welfare facilities")

welfare facilities


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Adequate changing rooms/locker,584,2383,161,833,474,117,1802,6354
Adequate eating area,584,2368,161,833,474,122,1802,6344
adequate washing facilities,584,2503,161,851,474,122,1802,6497
does the workplace have adequate accessible toilets?,584,2279,161,833,474,122,1802,6255
providing drinking water,584,2503,161,851,474,122,1802,6497
All,2920,12036,805,4201,2370,605,9010,31947


In [453]:
tab(cp="worker accommodation")

worker accommodation


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Vietnam,All
QCleaned,,,,,,,
has the employer adequately prepared for emergencies in the accommodation?,584,2271,161,833,474,1802,6125
is the accommodation protected against fire?,584,2239,161,833,474,1802,6093
is the accommodation separate from the workplace (even though it may be in the same compound/industrial park)?,584,2242,156,833,474,1802,6091
All,1752,6752,478,2499,1422,5406,18309


In [452]:
for i in ["is the accommodation protected against noise?",
          "is the accommodation protected against disease carrying animals or insects?",
          "is the accommodation adequately ventilated?",
          "is the accommodation adequately protected against heat, cold or dampness?",
          "is the accommodation adequately lit?",
          "does the accommodation offer workers adequate privacy?",
          "does the accommodation have laundry facilities?",
          "does the accommodation have enough safe water?",
          "does the accommodation have adequate toilets, showers, sewage and garbage disposal systems?",
          "does the accommodation have adequate eating and living areas?",
          "does the accommodation have adequate cooking facilities?",
          "are trained first aid personnel and adequate first aid supplies available in the accommodation?",
          "adequate space or minimum requirement"]:
    discard(i)

In [458]:
tab(cp="worker protection")

worker protection


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are appropriate safety warnings posted in the workplace?,584,2503,161,833,474,119,1802,6476
"are electrical wires, switches and plugs properly installed, grounded, and maintained?",584,2503,161,838,474,122,1802,6484
are proper guards installed and maintained on all dangerous machines and equipment?,584,2503,161,833,474,119,1802,6476
are workers effectively trained on occupational health and safety?,584,2286,161,833,474,119,1802,6259
boiler or steam operator certified,0,2434,0,834,0,122,1802,5192
do workers have suitable chairs?,584,2321,161,833,474,122,1802,6297
does the employer provide workers with all necessary personal protective clothing and equipment?,584,2503,161,851,474,122,1802,6497
prevent workers away from dangerous workplaces,584,2503,161,851,474,0,1802,6375
All,4088,19556,1127,6706,3318,845,14416,50056


In [455]:
for i in ["do the technicians for electrical installations and boiler operators have the legally required license/ certification?",
          "are workers protected against falls from heights?",
          "are all workers trained and obliged to use the personal protective equipment that is provided?"]:
    discard(i)
    ##items only asksed in one to three countries

In [457]:
df.drop(df[(df.QCleaned=="boiler or steam operator certified") & (df.Country=="Bangladesh")].index,inplace=True)
#357 deleted. Asked in Bangladesh only since 2018

In [463]:
tab(cp="working environment")

working environment


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are noise levels acceptable?,584,2371,161,833,474,122,1802,6347
is the workplace adequately lit?,584,2443,161,833,474,122,1802,6419
is the workplace clean and tidy?,584,2271,161,0,474,119,1802,5411
temperature and ventilation acceptable,584,2406,161,834,474,122,1802,6383
All,2336,9491,644,2500,1896,485,7208,24560


In [460]:
discard("is the workplace adequately ventilated?")
#1802 deleted; only in Vietnam

In [462]:
df.drop(df[(df.QCleaned=="is the workplace clean and tidy?") & (df.Country=="Indonesia")].index,inplace=True)
#480 deleted, asked in Indonesia only in some years

In [467]:
df.to_stata("June23items.dta",version=118)